In [ ]:
from hedge_stand import HedgeST,HedgeSTBoll
from datetime import date, datetime
from demeter import TokenInfo, PoolBaseInfo, Runner, Strategy, Asset, AccountStatus, BuyAction, SellAction, RowData, \
    ChainType
import optunity
import optunity.metrics
from decimal import Decimal
import pandas as pd
import  matplotlib.pylab as plt
# import logging 
# from logging import handlers
from load_data import pool_id_1_eth_u_500

from strategy_ploter import  plot_position_return_decomposition

%matplotlib inline

def backtest_boll(a, hedge_spread_split,hedge_spread_rate,alpha,boll_period_n):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    ema_max_spread_rate = round(0.2,4)

    eth = TokenInfo(name="eth", decimal=18)
    usdc = TokenInfo(name="usdc", decimal=6)
    pool = PoolBaseInfo(usdc, eth, 0.05, usdc)

    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Runner(pool)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeSTBoll(a=decimal_a,hedge_spread_split = decimal_hedge_spread_split,hedge_spread_rate=decimal_hedge_spread_rate,alpha=alpha,ema_max_spread_rate=ema_max_spread_rate,period_n=int(boll_period_n))
    runner_instance.set_assets([Asset(usdc, 10000)])
    save_path = f"../demeter/data/ETH/{pool_id_1_eth_u_500}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                pool_id_1_eth_u_500,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)

    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance


def backtest_instance(a, hedge_spread_split,hedge_spread_rate,alpha):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    alpha = round(alpha,4)
    # ema_max_spread_rate = round(ema_max_spread_rate,4)

    eth = TokenInfo(name="eth", decimal=18)
    usdc = TokenInfo(name="usdc", decimal=6)
    pool = PoolBaseInfo(usdc, eth, 0.05, usdc)

    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Runner(pool)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeST(decimal_a,decimal_hedge_spread_split,decimal_hedge_spread_rate,alpha)
    runner_instance.set_assets([Asset(usdc, 10000)])
    save_path = f"../demeter/data/ETH/{pool_id_1_eth_u_500}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                pool_id_1_eth_u_500,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)

    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance





In [ ]:

def generate_dfmerge(runner_instance):
    e = runner_instance.strategy.e

    total_net_value = runner_instance.final_status.net_value
        
    final_total_usdc_value = total_net_value + runner_instance.strategy.e.df['total'].iloc[-1]


    final_price = runner_instance.final_status.price
    # if net_value_base == 'USDC':
    profit_usdc = round(final_total_usdc_value- runner_instance.strategy.init_total_usdc,3)
    profit_rate_usdc = round(profit_usdc / runner_instance.strategy.init_total_usdc,3)
    # else:
    profit_eth = round(final_total_usdc_value/final_price - runner_instance.strategy.init_total_symbol,3)
    profit_rate_eth = round(profit_eth / runner_instance.strategy.init_total_symbol,3)

    print(f"USDC profit: {profit_usdc}, profit_rate: {profit_rate_usdc}")

    print(f"ETH profit: {profit_eth}, profit_rate: {profit_rate_eth}")

    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']

    df_merge.set_index('timestamp', inplace=True)

    df_merge['total_value_profit_rate'] = df_merge['total_value'] / runner_instance.strategy.init_total_usdc


    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge['total_value_eth_profit_rate'] = df_merge['total_value_eth'] / runner_instance.strategy.init_total_symbol

    df_merge_f = df_merge.apply(pd.to_numeric, downcast='float')
   
    df_merge_f['net_value_rate'] = df_merge_f['net_value'] / df_merge_f['net_value'].iloc[0]

    df_merge_f['price_rate'] = df_merge_f['price'] / df_merge_f['price'].iloc[0]
    return df_merge_f

In [ ]:
pd.options.plotting.backend = "plotly"

CHAIN_ID = 'ETH'
NET_VALUE_BASE = 'ETH'
RUNNING_TIME = 1
SEND_NOTICE = False
str_date_start = '2022-9-1'
str_date_end = '2022-10-31'

DATE_START = datetime.strptime(str_date_start, "%Y-%m-%d").date()

DATE_END = datetime.strptime(str_date_end, "%Y-%m-%d").date()

# A1 test

In [4]:
a = 1.17
hedge_spread_split = 2.88
hedge_spread_rate = 0.86
alpha = -1
ema_max_spread_rate=0.027
period_n = 6

instance=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n)

df_merge = generate_dfmerge(instance)

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

plot = df_merge[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

plot.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n}.png")

plot.show()

 12%|████████████▏                                                                                            | 20498/177120 [01:17<09:48, 266.07it/s]


KeyboardInterrupt: 

# A2 test

In [ ]:
a = 1.17
hedge_spread_split = 2.88
hedge_spread_rate = 0.86
alpha = 0.037
ema_max_spread_rate=0.027
period_n2 = 4

instance2=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n2)

df_merge2 = generate_dfmerge(instance2)

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

plot2 = df_merge2[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

plot2.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n2}.png")

plot2.show()

# a3 ema hedge

In [ ]:
a = 1.17
hedge_spread_split = 2.88
hedge_spread_rate = 0.86
alpha3 = 0.037
ema_max_spread_rate2=0.2

instance3=backtest_instance(a,hedge_spread_split,hedge_spread_rate,alpha3)

df_merge3 = generate_dfmerge(instance3)



plot3 = df_merge3[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

plot3.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha3}.png")

plot3.show()